In [1]:
import json
import pandas as pd
import re

In [3]:
with open('uniprot_json.json') as uniprot_json:
    uniprot_data = json.load(uniprot_json)


In [4]:
print(uniprot_data.keys())

dict_keys(['results'])


There is only one key and that is results

In [5]:
protein_access = uniprot_data['results']

In [6]:
print(protein_access[0])

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'A0A075TMP8', 'uniProtkbId': 'PATI_PENEN', 'entryAudit': {'firstPublicDate': '2018-12-05', 'lastAnnotationUpdateDate': '2022-12-14', 'lastSequenceUpdateDate': '2014-10-29', 'entryVersion': 31, 'sequenceVersion': 1}, 'annotationScore': 5.0, 'organism': {'scientificName': 'Penicillium expansum', 'commonName': 'Blue mold rot fungus', 'taxonId': 27334, 'lineage': ['Eukaryota', 'Fungi', 'Dikarya', 'Ascomycota', 'Pezizomycotina', 'Eurotiomycetes', 'Eurotiomycetidae', 'Eurotiales', 'Aspergillaceae', 'Penicillium']}, 'proteinExistence': '1: Evidence at protein level', 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000303', 'source': 'PubMed', 'id': '25120234'}], 'value': 'Cytochrome P450 monooxygenase patI'}, 'ecNumbers': [{'evidences': [{'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '30680886'}], 'value': '1.-.-.-'}]}, 'alternativeNames': [{'fullName': {'evidences': [{

In [6]:
protein_access[0].keys()

dict_keys(['entryType', 'primaryAccession', 'uniProtkbId', 'entryAudit', 'annotationScore', 'organism', 'proteinExistence', 'proteinDescription', 'genes', 'comments', 'features', 'keywords', 'references', 'uniProtKBCrossReferences', 'sequence', 'extraAttributes'])

chcem for cyklus ktory mi prejde cez vsetky proteiny kde potom zoberiem key primaryAccession lebo tam mam ID a dalej zoberiem key 'coments' a tam mam tie udaje o reakciach

In [7]:
print(len(protein_access))

1726


We have 1726 reviewed entries in this json file

In [8]:
data = []

for i in range(len(protein_access)):
    id = protein_access[i]['primaryAccession']
    comments = protein_access[i]['comments']
    reaction_found = False

    for j in range(len(comments)):
        if comments[j]['commentType'] == "CATALYTIC ACTIVITY":
            reaction = comments[j]['reaction']['name']
            reactionSplit = reaction.split(' = ')
            substrates = reactionSplit[0].strip()
            products = reactionSplit[1].strip()
            
            substratesSplit = substrates.split(' + ')
            productsSplit = products.split(' + ')
            participants = substratesSplit + productsSplit
            
            reducing_agent = '' 
            new_substrates = []
            new_products = []
            for sub in productsSplit:
                if sub.strip() not in ['H(+)', 'H2O', 'AH2', '2 H(+)', '2 H2O','3 H(+)', '3 H2O', '4 H(+)', '4 H2O', '6 H2O', 'oxidized [NADPH--hemoprotein reductase]', 
                           'oxidized [2Fe-2S]-[ferredoxin]', 'oxidized [adrenodoxin]', '2 oxidized [NADPH--hemoprotein reductase]', '6 oxidized [NADPH--hemoprotein reductase]', 
                           '2 oxidized [2Fe-2S]-[ferredoxin]', '2 oxidized [adrenodoxin]', '6 oxidized [2Fe-2S]-[ferredoxin]', '6 oxidized [adrenodoxin]',
                           'A', 'NAD(+)', 'NADP(+)', 'NADPH', 'NADH',  '2 NADP(+)', '2 NADPH', '2 NADH', 'CO2', '2 CO2', '3 CO2',  '3 oxidized [NADPH--hemoprotein reductase]', 
                           '3 oxidized [2Fe-2S]-[ferredoxin]', '3 oxidized [adrenodoxin]', '4 oxidized [NADPH--hemoprotein reductase]', 
                           '4 oxidized [2Fe-2S]-[ferredoxin]', '4 oxidized [adrenodoxin]', 'oxidized [NADH--hemoprotein reductase]', '2 oxidized [flavodoxin]',  'oxidized 2[4Fe-4S]-[ferredoxin]']:
                    new_products.append(sub.strip())
            
            for sub in substratesSplit:
                if 'reduced [' in sub:
                    reducing_agent = sub.strip()[0:9] + sub.strip()[9:-1] + sub.strip()[-1:]
                elif '2 reduced ferredoxin [iron-sulfur] cluster' in sub:
                    reducing_agent =  '2 reduced ferredoxin [iron-sulfur] cluster'
                elif 'reduced 2[4Fe-4S]-[ferredoxin]' in sub:
                    reducing_agent =  'reduced 2[4Fe-4S]-[ferredoxin]'
                elif sub in ['NADPH', 'NADP(+)', 'NAD(+)', 'NADH', '2 NADPH', '3 NADPH']:
                    reducing_agent= sub.strip()
                elif sub not in ['O2', 'H(+)', 'AH2', 'H2O', 'H2O2', 'A', '2 O2', '2 H(+)', '2 AH2', '2 H2O', '2 H2O2', 
                                 '2 A', '3 O2', '3 H(+)', '3 AH2', '3 H2O', '3 H2O2', '3 A', '4 H(+)', '5 H(+)', '6 H(+)', '4 O2']:
                    new_substrates.append(sub.strip())
                    
            crossRef = comments[j]['reaction'].get('reactionCrossReferences', [])
            
            rheaID = ''
            chebis = []
            participant_chebis = {}
            for ref in crossRef:
                if ref['database'] == 'Rhea':
                    rheaID = ref['id']
                    break
            for ref in crossRef:
                if ref['database'] == 'ChEBI':
                    chebis.append(ref['id'])

                

            # create a dict that maps the participants of all reactions to their CHEBI IDs
            
            data.append({'id': id, 'reaction': reaction, 'rheaID': rheaID,  
                         'reducing_agent': reducing_agent, 'substrates': new_substrates, 
                         'products': new_products,'participants': participants, 'chebis': chebis})
            reaction_found = True


    # before adding this 2929 records in dataframe, after adding this 3781
    if not reaction_found:
        data.append({'id': id, 'reaction': None, 'rheaID': None, 'reducing_agent': None, 'substrates': None, 
                         'products': None,'participants': None, 'chebis': None})

df = pd.DataFrame(data)

df.to_csv('reactionsfromjson.csv', index=False)
df.to_excel('reactionFromJSONexcel.xlsx', index=False)

ModuleNotFoundError: No module named 'openpyxl'